In [1]:
from pre_process import *
from numpy import log
import time
from numpy import sqrt
import matplotlib.pyplot as plt

def fSR(list_2):
    '''
    calculate the fSR used in the GOR III Aalgorithm
    :param list_2:  list from the reading of stride or dssp
    :return: a 2 d dictionary
    '''
    result = {}
    fS = {}
    fR = {}
    for i in list_2:
        # calculate fSR
        if i[4] in result.keys():
            if i[3] in result[i[4]].keys():
                result[i[4]][i[3]] += 1
            else:

                result[i[4]][i[3]] = 1
        else:
            temp = {}
            temp[i[3]] = 1
            result[i[4]] = temp
        # calcuate fS
        if i[4] in fS.keys():
            fS[i[4]] += 1
        else:
            fS[i[4]] = 1
        # calculate fR
        if i[3] in fR.keys():
            fR[i[3]] += 1
        else:
            fR[i[3]] = 1

    return result, fS, fR

In [2]:
def read_txt(filename):
    '''
    :use to read structured txt file seperated with ' '
    :param filename: the position of the file
    :return: a two demension list
    '''
    f = open(filename)
    result = []
    for i in f.readlines():
        temp = i.strip().split()
        result.append(temp)
    f.close()
    return result



# print(len(dssp),len(stride),len(cath))


f = open("./txtfile/amino_dict.txt")
amino_dict = {}
for i in f.readlines():
    temp = i.strip().split()
    amino_dict[temp[0].upper()] = temp[1]
f.close()

f = open("./txtfile/stru_dict.txt")
stru_dict = {}
for i in f.readlines():
    temp = i.strip().split()
    stru_dict[temp[0].upper()] = temp[1]
f.close()



def pre_process(file, newfile):
    '''
    do preprocessing, only for dssp_info and stride_info
    do with uncertain values
    :param list_2: 2-d list
    :return: 1-d list
    '''
    list_2 = read_txt(file)

    fw = open(newfile, 'w')
    for i in list_2:
        if i[3].upper() in amino_dict.keys():
            i[3] = amino_dict[i[3].upper()]
        else:
            i[3] = "?"
        if i[4].upper() in stru_dict.keys():
            i[4] = stru_dict[i[4].upper()]
        s = ''
        for j in i:
            s += j + ' '
        s += '\n'
        fw.write(s)

    return list_2

In [3]:
# transfer dssp_predict.txt file into format like fasta

def tofasta(file,newfile):
    f = open(file)
    f2 = open(newfile,'w')
    t = f.readline().strip().split()
    f2.write(t[0] + ' ' + t[1] + ' ' + t[2])
    tlist = []
    t2list = []
    tlist.append(t[3])
    t2list.append(t[4])
    for i in f.readlines():
        temp = i.strip().split()
        if temp[0] == t[0] and temp[1] == t[1]:
            tlist.append(temp[3])
            t2list.append(temp[4])
            pos = temp[2]
        else:
            f2.write('-' + pos)
            f2.write('\n')
            f2.write(''.join(tlist))
            f2.write('\n')
            f2.write(''.join(t2list))
            f2.write('\n')
            tlist = []
            t2list = []
            t = i.strip().split()
            f2.write(t[0] + ' ' + t[1] + ' ' + t[2])
            tlist.append(t[3])
            t2list.append(t[4])

    f2.write('\n')
    f2.write(''.join(tlist))
    f2.write('\n')
    f2.write(''.join(t2list))
    f2.write('\n')

    f.close()
    f2.close()

tofasta('./txtfile/dssp.txt','./txtfile/dssp_protein')
tofasta('./txtfile/stride.txt','./txtfile/stride_protein')

In [4]:
dssp2 = pre_process("./txtfile/dssp_info.txt", "./txtfile/dssp.txt")
stride = pre_process("./txtfile/stride_info.txt", "./txtfile/stride.txt")

fsr, fs, fr = fSR(dssp2)
fsr2, fs2, fr2 = fSR(stride)

total = fs['C'] + fs['E'] + fs['C']
total2 = fs2['C'] + fs2['E'] + fs2['C']

In [5]:
f = open("./txtfile/dssp_protein")

list = f.readlines()
list2 = []
for i in range(0, len(list), 3):
    temp = []
    temp.append(list[i + 1].strip())
    temp.append(list[i + 2].strip())
    temp.append(list[i].strip())
    list2.append(temp)
f.close()
# print(list2)
# dealing the statistics
fsrm = {}
# initialize
amino = []
for i in amino_dict.keys():
    amino.append(amino_dict[i])
amino.insert(0,"?")
# print(amino)
for i in ['C', 'E', 'H']:
    temp3 = {}
    for j in amino:
        temp2 = {}
        for k in amino:
            temp = {}
            for n in range(1, 9):
                temp[n] = 1

            temp2[k] = temp

        temp3[j] = temp2
    fsrm[i] = temp3

for i in list2:
    for j in range(len(i[0]) - 8):
        for k in range(1, 9):
            fsrm[i[1][j]][i[0][j]][i[0][k + j]][k] += 1

In [6]:
def gor3(alist):
    result = []
    for i in range(len(alist)):

        helix = log(fsr['H'][alist[i]] / (fsr['C'][alist[i]] + fsr['E'][alist[i]])) + log(
            (fs['E'] + fs['C']) / fs['H'])
        coil = log(fsr['C'][alist[i]] / (fsr['H'][alist[i]] + fsr['E'][alist[i]])) + log(
            (fs['H'] + fs['E']) / fs['C'])
        sheet = log(fsr['E'][alist[i]] / (fsr['C'][alist[i]] + fsr['H'][alist[i]])) + log(
            (fs['H'] + fs['C']) / fs['E'])

        for j in range(-8, 9):
            t = i + j
            if t > 0 and t < len(alist):
                if j != 0:
                    helix += log(fsrm['H'][alist[i]][alist[t]][abs(j)] / (
                        fsrm['E'][alist[i]][alist[t]][abs(j)] + fsrm['C'][alist[i]][alist[t]][abs(j)])) + log(
                        (fsr['E'][alist[i]] + fsr['C'][alist[i]]) / fsr['H'][alist[i]])

                    coil += log(fsrm['C'][alist[i]][alist[t]][abs(j)] / (
                        fsrm['E'][alist[i]][alist[t]][abs(j)] + fsrm['H'][alist[i]][alist[t]][abs(j)])) + log(
                        (fsr['E'][alist[i]] + fsr['H'][alist[i]]) / fsr['C'][alist[i]])
                    sheet += log(fsrm['E'][alist[i]][alist[t]][abs(j)] / (
                        fsrm['H'][alist[i]][alist[t]][abs(j)] + fsrm['C'][alist[i]][alist[t]][abs(j)])) + log(
                        (fsr['H'][alist[i]] + fsr['C'][alist[i]]) / fsr['E'][alist[i]])

        if coil == max(sheet, coil, helix):
            result.append('C')
        elif sheet == max(sheet, coil, helix):
            result.append('E')

        else:
            result.append('H')

    return ''.join(result)

In [7]:
f = open("./txtfile/dssp_predict.txt", 'w')

predict = ""
right = ""

for i in list2:
    # leave one out procedure

    for j in range(len(i[0]) - 8):
        for k in range(1, 9):
            fsrm[i[1][j]][i[0][j]][i[0][k + j]][k] -= 1


    f.write(i[2] + '\n')
    f.write(i[0] + '\n')
    # write the right structure
    f.write(i[1] + '\n')
    right += i[1]

    # write the predict structure
    temp = gor3(i[0])
    predict += temp
    f.write(temp + '\n')

    # restore

    for j in range(len(i[0]) - 8):
        for k in range(1, 9):
            fsrm[i[1][j]][i[0][j]][i[0][k + j]][k] += 1


f.close()

enum = 0
hnum = 0
cnum = 0
for i in right:
    if i == 'C':
        cnum += 1
    elif i == 'E':
        enum += 1
    else:
        hnum += 1

ewrong = 0
hwrong = 0
cwrong = 0
num = 0
for i in range(len(predict)):
    if predict[i] == right[i]:
        num += 1
    elif right[i] == 'E':
        ewrong += 1
    elif right[i] == 'C':
        cwrong += 1
    else:
        hwrong += 1

print(num / len(predict))


def mcc(predict, right, stru):
    TP, FP, FN, TN = 0, 0, 0, 0
    for i in range(len(predict)):
        if predict[i] == stru:
            if right[i] == stru:
                TP += 1
            else:
                FP += 1
        else:
            if right[i] == stru:
                FN += 1
            else:
                TN += 1
    MCC = (TP * TN - FP * FN) / sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    TRP = TP / (TP + FN)
    SPC = TN / (FP + TN)

    print(TP, FP, TN, FN)

    return MCC, TRP, SPC

0.5574512193413735


In [14]:
f = open("./txtfile/stride_protein")

list = f.readlines()
list2 = []
for i in range(0, len(list), 3):
    temp = []
    temp.append(list[i + 1].strip())
    temp.append(list[i + 2].strip())
    temp.append(list[i].strip())
    list2.append(temp)
f.close()
# print(list2)
# dealing the statistics
fsrm2 = {}
# initialize
amino = []
for i in amino_dict.keys():
    amino.append(amino_dict[i])
amino.insert(0,"?")
# print(amino)
for i in ['C', 'E', 'H']:
    temp3 = {}
    for j in amino:
        temp2 = {}
        for k in amino:
            temp = {}
            for n in range(1, 9):
                temp[n] = 1

            temp2[k] = temp

        temp3[j] = temp2
    fsrm2[i] = temp3

for i in list2:
    for j in range(len(i[0]) - 8):
        for k in range(1, 9):
            fsrm2[i[1][j]][i[0][j]][i[0][k + j]][k] += 1

In [15]:
def gor32(alist):
    result = []
    for i in range(len(alist)):

        helix = log(fsr2['H'][alist[i]] / (fsr2['C'][alist[i]] + fsr2['E'][alist[i]])) + log(
            (fs2['E'] + fs2['C']) / fs2['H'])
        coil = log(fsr2['C'][alist[i]] / (fsr2['H'][alist[i]] + fsr2['E'][alist[i]])) + log(
            (fs2['H'] + fs2['E']) / fs2['C'])
        sheet = log(fsr2['E'][alist[i]] / (fsr2['C'][alist[i]] + fsr2['H'][alist[i]])) + log(
            (fs2['H'] + fs2['C']) / fs2['E'])

        for j in range(-8, 9):
            t = i + j
            if t > 0 and t < len(alist):
                if j != 0:
                    helix += log(fsrm2['H'][alist[i]][alist[t]][abs(j)] / (
                        fsrm2['E'][alist[i]][alist[t]][abs(j)] + fsrm2['C'][alist[i]][alist[t]][abs(j)])) + log(
                        (fsr2['E'][alist[i]] + fsr2['C'][alist[i]]) / fsr2['H'][alist[i]])

                    coil += log(fsrm2['C'][alist[i]][alist[t]][abs(j)] / (
                        fsrm2['E'][alist[i]][alist[t]][abs(j)] + fsrm2['H'][alist[i]][alist[t]][abs(j)])) + log(
                        (fsr2['E'][alist[i]] + fsr2['H'][alist[i]]) / fsr2['C'][alist[i]])
                    sheet += log(fsrm2['E'][alist[i]][alist[t]][abs(j)] / (
                        fsrm2['H'][alist[i]][alist[t]][abs(j)] + fsrm2['C'][alist[i]][alist[t]][abs(j)])) + log(
                        (fsr2['H'][alist[i]] + fsr2['C'][alist[i]]) / fsr2['E'][alist[i]])

        if coil == max(sheet, coil, helix):
            result.append('C')
        elif sheet == max(sheet, coil, helix):
            result.append('E')

        else:
            result.append('H')

    return ''.join(result)

In [16]:
f = open("./txtfile/stride_predict.txt", 'w')

predict = ""
right = ""

for i in list2:
    # leave one out procedure

    for j in range(len(i[0]) - 8):
        for k in range(1, 9):
            fsrm2[i[1][j]][i[0][j]][i[0][k + j]][k] -= 1


    f.write(i[2] + '\n')
    f.write(i[0] + '\n')
    # write the right structure
    f.write(i[1] + '\n')
    right += i[1]

    # write the predict structure
    temp = gor32(i[0])
    predict += temp
    f.write(temp + '\n')

    # restore

    for j in range(len(i[0]) - 8):
        for k in range(1, 9):
            fsrm2[i[1][j]][i[0][j]][i[0][k + j]][k] += 1


f.close()

enum = 0
hnum = 0
cnum = 0
for i in right:
    if i == 'C':
        cnum += 1
    elif i == 'E':
        enum += 1
    else:
        hnum += 1

ewrong = 0
hwrong = 0
cwrong = 0
num = 0
for i in range(len(predict)):
    if predict[i] == right[i]:
        num += 1
    elif right[i] == 'E':
        ewrong += 1
    elif right[i] == 'C':
        cwrong += 1
    else:
        hwrong += 1

print(num / len(predict))

KeyError: '?'

Cath

In [ ]:
def read_fasta(filename):
    f = open(filename)
    proteins = []
    d = f.read().split(">")

    # deal with the  "" before the first ""
    d.pop(0)
    for i in d:
        t = i.splitlines()
        # get rid of the line of the protein name
        t.pop(0)
        s = "".join(t)
        proteins.append(s)
    return proteins

In [ ]:
import matplotlib.pyplot as  plt
from numpy import sqrt

f = open("./txtfile/cath_info.txt")

fw = open("./txtfile/cath.txt", 'w')

dict = {}
dict['Alpha'] = 'A'
dict['Beta'] = 'B'
dict["Alpha/beta"] = 'AB'
dict["None"] = 'N'

for i in f.readlines():
    t = i.strip().split()

    t[2] = dict[t[2]]

    fw.write(t[0] + ' ')
    fw.write(t[1] + ' ')
    fw.write(t[2] + '\n')

fw.close()
f.close()

f = open("./txtfile/cath.txt")
right = []
for i in f.readlines():
    t = []
    t.append(i.strip().split()[0])
    t.append(i.strip().split()[2])
    right.append(t)

f.close()

f = open("./txtfile/dssp_predict.txt")
list = f.readlines()
list2 = []
for i in range(0, len(list), 4):
    t = []
    t.append(list[i].strip().split()[0])
    t.append(list[i].strip().split()[2])
    t.append(list[i + 3].strip())
    t.append(list[i + 2].strip())
    t.append(list[i + 1].strip())
    list2.append(t)


# print(list2)

def family_predict(alist):
    cnum = 1
    enum = 1
    hnum = 1
    for i in alist:
        if i == 'C':
            cnum += 1
        elif i == 'E':
            enum += 1
        else:
            hnum += 1

    return (hnum-enum)/(len(alist)-cnum)






result = []
for i in list2:
    t = []
    t.append(family_predict(i[2]))
    t.append(i[0])
    t.append(i[2])
    t.append(i[3])
    t.append(i[4])
    result.append(t)

#print(result)


def q3(right, predict):
    num = 0
    for i in range(len(right)):
        if right[i] == predict[i]:
            num += 1
    return num / len(right)


def mcc(predict, right, stru):
    TP, FP, FN, TN = 1, 1, 1, 1
    for i in range(len(predict)):
        if predict[i] == stru:
            if right[i] == stru:
                TP += 1
            else:
                FP += 1
        else:
            if right[i] == stru:
                FN += 1
            else:
                TN += 1
    MCC = (TP * TN - FP * FN) / sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    TRP = TP / (TP + FN)
    SPC = TN / (FP + TN)

    return MCC


temp = []
accuracy = 0
for i in range(len(right)):
    t = []
    t.append(result[i][0])
    t.append(right[i][1])

    if result[i][0] > 0.45:
        t.append('A')
    elif result[i][0] < -0.25:
        t.append('B')
    else:
        t.append('AB')

    t.append(right[i][0])

    t.append(result[i][2])
    t.append(result[i][3])
    t.append(result[i][4])
    t.append(q3(result[i][3], result[i][2]))
    t.append(mcc(result[i][3], result[i][2], 'H'))
    t.append(mcc(result[i][3], result[i][2], 'E'))
    t.append(mcc(result[i][3], result[i][2], 'C'))
    temp.append(t)

    if t[1] == t[2]:
        accuracy += 1

print("accuracy = ", accuracy / len(right))

for i in temp:
    if i[1] == 'A':
        plt.plot([1], [i[0]], 'ro')
    elif i[1] == 'B':
        plt.plot([2], [i[0]], 'bo')
    else:
        plt.plot([3], [i[0]], 'go')

# plt.show()





temp.sort()
f = open("./txtfile/cathinfo2.csv", 'w')
f.write('H-E,')
f.write('actual family,')
f.write('predict family,')
f.write('name,')
f.write('predict structure,')
f.write('actual structure,')
f.write("amino,")
f.write('q3,')
f.write('mccH,')
f.write('mccE,')
f.write('mccC,')
f.write('\n')

for i in temp:
    f.write(str(i[0]) + ',')
    f.write(str(i[1]) + ',')
    f.write(str(i[2]) + ',')
    f.write(str(i[3]) + ',')
    f.write(str(i[4]) + ',')
    f.write(str(i[5]) + ',')
    f.write(str(i[6]) + ',')
    f.write(str(i[7]) + ',')
    f.write(str(i[8]) + ',')
    f.write(str(i[9]) + ',')
    f.write(str(i[10]) + '\n')

#print(temp)

In [ ]:
from read_fasta import read_fasta
from gor3 import gor3




# GHR--G-D -> GHRGD
def tostr(alist):
    result = ''
    for i in alist:
        if i != '-':
            result += i
    return result


# GHRGD -->EEE--C-D
def reverse(alist1, alist):
    result = ''

    # index of alist
    j = 0
    for i in alist1:
        if i != '-':
            result += alist[j]
            j += 1
        else:
            result += '-'
    return result


# print(reverse(seq[1][start:end],tostr(seq[1][start:end])))

# print(seq[1][start:end])
# print(tostr(seq[1][start:end]))





astru_right="CCCCCCCCCCCCCCHHHHHHHHHHHHHHCCCCEEEEEEEECCCCCEEEEEEECCCCCCCCEEEEEECCCCCCHHHHHHHHHHHHHHHHHCCCCHHHHHHHHHCEEEEECCCCHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCHHHCCCCCCCCCCCECCCCCCCECCCCCCCCHHHHHHHHHHHHHCCEEEEEEEEECCCEEEECCCCCCCCCCCHHHHHHHHHHHHHHHHHHHCCCCEEEEHHHHCCCCCCCHHHHHHHCCCCEEEEEEECCCCCCHHHCCHHHHHHHHHHHHHHHHHHHHHHHHC"
bstru_right ='CCCCECECCCCCECECCCEEEEEECCHHHCCCEEEEEECCEEEEEEEEECCCCCCCCCCEEEEECCCCCCCCECECCCCEEEEECCCCCCCCCCECEECCCCECCECEEECCCCCCCCCCCHHHCEEEEECECCCCCCEEEEEECCCEEECEEECCCCCCCCEEECCCCECCEEEEEECCC'
cstru_right = 'CCCCCCCCCCCCCCCCCCCCHHHHHHHHHCHHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHHHHHHHHHHCECCCCCCCCCCCHHHHHHHHHHHCCHHHHHHHHHHHHHCCCCCEEEEEEEECCCCEEEEEEEECCCECCCCCCEEEEEEECCHHHCHHHHCCCHHHHHHHHHHHECHHHHHHHHHHHCCC'
dstru_right = 'CCCCCECCCECCCECCCCCCCEEEEEEECCEEEEEEEHHHHHHHHHHHHHHHHHHHCCCCEECCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHEEECCCCEEEECCCCCHHHHHHHHCCCCCCHHHHHHHHHHHCCCCC'
estru_right ='CCCCCCCCCCCHHHCCCCHHHHHHHHHHHHHHHHHHHCCCHHHHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHCHHHHCCCC'
fstru_right = 'CCCCCCCCCCCCCCCHHHHHHHHHHHHCCCCHHHHHHHCEECCCCEECCCCCCCEEEECCCCCHHHHCCCCCCCCCCCCCCCCC'
gstru_right = 'CCHHHHHHHCCCCCCCCCCCCCCCCCHHHHHHHCCCCHHHHHHHHCCCCCCCCCCCCHHHHCCCCCCCCCCCHHHHHHHHHHHHCC'
'''
f = open("./fasta/6sequence.txt",'w')
f.write(tostr(read_fasta("./fasta/a_align.fasta")[0]))
f.write("\n")
f.write(tostr(read_fasta("./fasta/b_align.fasta")[0]))
f.write("\n")
f.write(tostr(read_fasta("./fasta/c_align.fasta")[0]))
f.write("\n")
f.write(tostr(read_fasta("./fasta/d_align.fasta")[0]))
f.write("\n")

f.write(tostr(read_fasta("./fasta/e_align.fasta")[0]))
f.write("\n")

f.write(tostr(read_fasta("./fasta/f_align.fasta")[0]))
f.write("\n")
f.write(astru_right+"\n")
f.write(bstru_right+"\n")
f.write(cstru_right+"\n")
f.write(dstru_right+"\n")
f.write(estru_right+"\n")
f.write(fstru_right+"\n")
'''

def q3(list1,list2):
    right = 0
    for i in range(len(list1)):
        if list1[i] == list2[i]:
            right+=1
    return right/len(list1)




def align_predict(filename):
    seq = read_fasta(filename)
    for i in range(len(seq[0])):
        if seq[0][i] != '-':
            start = i
            break

    for i in range(len(seq[0]) - 1, 0, -1):
        if seq[0][i] != '-':
            end = i + 1
            break

    result = []
    for i in range(len(seq)):
        t1 = seq[i][start:end]
        t2 = gor3(tostr(t1))
        result.append(reverse(t1, t2))
    '''
    for i in result:
        print(i)

    '''
    stru_pred = ''
    for i in range(len(result[0])):
        if result[0][i] != '-':
            temp = [result[j][i] for j in range(len(result))]

            t1 = temp.count('E')
            t2 = temp.count('C')
            t3 = temp.count('H')

            if t1 == max(t1, t2, t3):
                stru_pred += 'E'
            elif t2 == max(t1, t2, t3):
                stru_pred += 'C'
            else:
                stru_pred += 'H'
    return stru_pred


'''
t = align_predict("./fasta/a_align1.fasta")
print(t)
print(astru_right)
print(q3(t,astru_right))
print(q3(gor3(tostr(read_fasta("./fasta/a_align1.fasta")[0])),astru_right))

'''





t = align_predict("./fasta/b_align3.fasta")
print(t)
print(bstru_right)
print(q3(t,bstru_right))
b = read_fasta("./fasta/b_align3.fasta")
print(q3(gor3(tostr(b[0])),bstru_right))
print(len(b))



'''
t = align_predict("./fasta/c_align2.fasta")
print(t)
print(cstru_right)
print(q3(t,cstru_right))
print(q3(gor3(tostr(read_fasta("./fasta/c_align1.fasta")[0])),cstru_right))

'''


'''
t = align_predict("./fasta/d_align.fasta")
print(t)
print(dstru_right)
print(q3(t,dstru_right))
print(q3(gor3(tostr(read_fasta("./fasta/d_align.fasta")[0])),dstru_right))
'''

'''
t = align_predict("./fasta/e_align1.fasta")
print(t)
print(estru_right)
print(q3(t,estru_right))
b = read_fasta("./fasta/e_align1.fasta")
print(q3(gor3(tostr(b[0])),estru_right))
print(len(b))

'''


'''
t = align_predict("./fasta/f_align.fasta")
print(t)
print(fstru_right)
print(q3(t,fstru_right))
print(q3(gor3(tostr(read_fasta("./fasta/f_align.fasta")[0])),fstru_right))

'''